<h1 style="text-align: center;" markdown="1">TP2: Random Forest tuneo de parametros</h1>
![](../data/icon_properati-data.png)

In [1]:
from sklearn import svm
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV

import pandas as pd

# modules
import knn as knnlibrary

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
properties =knnlibrary.get_dataset()
properties.info()

['properati-AR-2016-12-01-properties-sell.csv', 'properati-AR-2016-05-01-properties-sell.csv', 'properati-AR-2016-06-01-properties-sell.csv', 'properati-AR-2016-08-01-properties-sell.csv', 'properati-AR-2017-02-01-properties-sell.csv', 'properati-AR-2017-08-01-properties-sell-six_months.csv', 'properati-AR-2016-04-01-properties-sell.csv', 'properati-AR-2016-11-01-properties-sell.csv', 'properati-AR-2015-02-01-properties-sell-six_months.csv', 'properati-AR-2015-12-01-properties-sell-six_months.csv', 'properati-AR-2016-03-01-properties-sell-six_months.csv', 'properati-AR-2017-01-01-properties-sell.csv', 'properati-AR-2016-10-01-properties-sell.csv', 'properati-AR-2016-07-01-properties-sell.csv', 'properati-AR-2015-07-01-properties-sell-six_months.csv', 'properati-AR-2016-09-01-properties-sell.csv', 'properati-AR-2017-07-01-properties-sell-six_months.csv']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 457560 entries, 0 to 80257
Data columns (total 27 columns):
id                      

In [28]:
# transformo el campo fecha
properties_2017 = properties[properties['created_on'] > '2017-0-0']

# filtro por CABA y GBA
properties_caba = properties_2017[(properties_2017['place_with_parent_names'].str.contains('Capital Federal') \
                             | properties_2017['place_with_parent_names'].str.contains('Bs.As. G.B.A.'))]

properties_caba = properties_caba[properties_caba['price'].notnull() & properties_caba['price'] > 0 & properties_caba['place_name'].notnull()]

# eliminamos propiedades con mas de 54 pisos
properties_caba = properties_caba[properties_caba['floor'] <= 60]
# eliminamos propiedades con mas de 9 pisos - ver analisis
properties_caba = properties_caba[properties_caba['rooms'] <=  10]
# eliminamos propiedades con mas de 2000 m2 de superficie cubierta - ver analisis
properties_caba = properties_caba[(properties_caba['surface_covered_in_m2'] <= 4000) & (properties_caba['surface_covered_in_m2'] >= 0)]
# eliminamos propiedades con mas de 2000 m2 de superficie cubierta - ver analisis
properties_caba = properties_caba[(properties_caba['surface_total_in_m2']<= 6000)  & (properties_caba['surface_total_in_m2'] >= 0)]
properties_caba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2958 entries, 31261 to 80073
Data columns (total 27 columns):
id                            2958 non-null object
created_on                    2958 non-null object
operation                     2958 non-null object
property_type                 2958 non-null object
place_name                    2958 non-null object
place_with_parent_names       2958 non-null object
country_name                  2958 non-null object
state_name                    2958 non-null object
geonames_id                   2485 non-null float64
lat-lon                       2791 non-null object
lat                           2791 non-null float64
lon                           2791 non-null float64
price                         2958 non-null float64
currency                      2958 non-null object
price_aprox_local_currency    2958 non-null float64
price_aprox_usd               2958 non-null float64
surface_total_in_m2           2958 non-null float64
surface_covere

In [29]:
# filtro columnas segun lo que hay en el dataset
attributes = ['created_on','property_type','place_name','place_with_parent_names',\
              'country_name','state_name','lat','lon','surface_total_in_m2','surface_covered_in_m2',\
              'floor','rooms', 'price']
properties_caba[attributes].info()
properties_caba_with_price_attributes = properties_caba[attributes]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2958 entries, 31261 to 80073
Data columns (total 13 columns):
created_on                 2958 non-null object
property_type              2958 non-null object
place_name                 2958 non-null object
place_with_parent_names    2958 non-null object
country_name               2958 non-null object
state_name                 2958 non-null object
lat                        2791 non-null float64
lon                        2791 non-null float64
surface_total_in_m2        2958 non-null float64
surface_covered_in_m2      2958 non-null float64
floor                      2958 non-null float64
rooms                      2958 non-null float64
price                      2958 non-null float64
dtypes: float64(7), object(6)
memory usage: 323.5+ KB


In [30]:
from sklearn.preprocessing import LabelEncoder

properties_caba_with_price_attributes.fillna('NaN', inplace=True)
# atributos categoricos
encoder = LabelEncoder()

properties_caba_with_price_attributes['property_type'] = encoder.fit_transform(properties_caba_with_price_attributes[['property_type']])
#properties_caba_with_price_attributes['operation'] = encoder.fit_transform(properties_caba_with_price_attributes[['operation']])
properties_caba_with_price_attributes['place_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['place_name']])
properties_caba_with_price_attributes['place_with_parent_names'] = encoder.fit_transform(properties_caba_with_price_attributes[['place_with_parent_names']])
properties_caba_with_price_attributes['country_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['country_name']])
properties_caba_with_price_attributes['state_name'] = encoder.fit_transform(properties_caba_with_price_attributes[['state_name']])
properties_caba_with_price_attributes['created_on'] = encoder.fit_transform(properties_caba_with_price_attributes[['created_on']])
# completamos valores nan
from sklearn.preprocessing import Imputer
imputer_mean = Imputer(missing_values='NaN', strategy='mean', axis=0)
properties_caba_with_price_attributes["lat"] = imputer_mean.fit_transform(properties_caba_with_price_attributes[["lat"]])
properties_caba_with_price_attributes["lon"] = imputer_mean.fit_transform(properties_caba_with_price_attributes[["lon"]])

properties_caba_with_price_attributes.head()

/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A

,created_on,property_type,place_name,place_with_parent_names,country_name,state_name,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,price
31261,0,1,15,38,0,1,-34.632319,-58.567871,90.0,68.0,3.0,3.0,145000.0
31270,0,0,36,50,0,2,-34.606879,-58.464805,73.0,34.0,2.0,2.0,930000.0
32137,1,1,73,28,0,1,-34.681743,-58.558447,37.0,37.0,4.0,2.0,90000.0
32143,1,1,15,38,0,1,-34.605972,-58.563086,83.0,79.0,23.0,4.0,145000.0
32162,1,1,2,59,0,3,-34.605446,-58.417212,55.0,55.0,1.0,4.0,110000.0


In [31]:
# separamos el set de train
from sklearn.model_selection import train_test_split
import datetime

now = datetime.datetime.now()

# separamos el train de traing para validarlo luego usando un 30% de los datos
X, y = properties_caba_with_price_attributes.iloc[:, properties_caba_with_price_attributes.columns != 'price'].values, properties_caba_with_price_attributes.iloc[:, properties_caba_with_price_attributes.columns == 'price'].values
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.2, random_state=200)
    
print(y_train)
    
len(X_test), len(X_train)

[[  50000.]
 [  97000.]
 [ 957260.]
 ..., 
 [ 450000.]
 [ 120000.]
 [ 165000.]]


(592, 2366)

In [32]:
# preprocesamiento
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

pipeline = make_pipeline(preprocessing.StandardScaler(), RandomForestRegressor(n_estimators=100))

In [33]:
# hiperparametros a tunear
print(pipeline.get_params())

hyperparameters = { 'randomforestregressor__max_depth': [10,30,50,70,90,100],
                    'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                    'randomforestregressor__min_samples_leaf': [5,10,15,20],
                    'randomforestregressor__min_samples_split': [5,10,15,20]}

{'randomforestregressor__oob_score': False, 'randomforestregressor__random_state': None, 'randomforestregressor__verbose': 0, 'randomforestregressor__max_depth': None, 'randomforestregressor__min_samples_split': 2, 'randomforestregressor__criterion': 'mse', 'randomforestregressor__warm_start': False, 'standardscaler__copy': True, 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False), 'standardscaler__with_std': True, 'randomforestregressor__min_impurity_split': None, 'memory': None, 'randomforestregressor__n_jobs': 1, 'randomforestregressor__n_estimators': 100, 'randomforestregressor__min_samples_leaf': 1, 'steps': [('standardscaler', St

In [34]:
# crossvalidation
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
# Fit and tune model
clf.fit(X_train, y_train.ravel())

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decr...mators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'], 'randomforestregressor__min_samples_split': [5, 10, 15, 20], 'randomforestregressor__max_depth': [10, 30, 50, 70, 90, 100], 'randomforestregressor__min_samples_leaf': [5, 10, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [35]:
# cuales son los mejores hiperparametros
print(clf.best_params_)

#Confirm model will be retrained
print(clf.refit)

{'randomforestregressor__max_features': 'auto', 'randomforestregressor__min_samples_split': 10, 'randomforestregressor__max_depth': 10, 'randomforestregressor__min_samples_leaf': 5}
True


In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt

y_pred = clf.predict(X_test)

print(r2_score(y_test, y_pred))
 
print(sqrt(mean_squared_error(y_test, y_pred)))

0.890383626675
108229.66311734897


In [40]:
model = RandomForestRegressor(n_estimators=3000, max_features='auto', max_depth=10,min_samples_leaf=5,min_samples_split=10 )
model.fit(X_train, y_train)

r2_score(y_test, model.predict(X_test))

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


0.8930850174490752

# Encuentro hiper-parametros usando los valores normalizados

In [37]:
# crossvalidation
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
# Fit and tune model
clf.fit(X_train_scaled, y_train.ravel())

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decr...mators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'], 'randomforestregressor__min_samples_split': [5, 10, 15, 20], 'randomforestregressor__max_depth': [10, 30, 50, 70, 90, 100], 'randomforestregressor__min_samples_leaf': [5, 10, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [38]:
# diferencia con los sets estandarizados
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

y_pred = clf.predict(X_test_scaled)

print(r2_score(y_test, y_pred))
 
print(sqrt(mean_squared_error(y_test, y_pred)))

0.88770368872
109544.69168553899


In [39]:
# cuales son los mejores hiperparametros
print(clf.best_params_)

#Confirm model will be retrained
print(clf.refit)

{'randomforestregressor__max_features': 'auto', 'randomforestregressor__min_samples_split': 10, 'randomforestregressor__max_depth': 100, 'randomforestregressor__min_samples_leaf': 5}
True


In [41]:
# Creando modelo con hiper-parametros encontrados
model_std = RandomForestRegressor(n_estimators=3000, max_features='auto', max_depth=100,min_samples_leaf=5,min_samples_split=10 )
model_std.fit(X_train_scaled, y_train)

r2_score(y_test, model_std.predict(X_test_scaled))

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.89312056160868725

# Prediction

In [42]:
# leemos set de test
test_df = pd.read_csv('../data/test/properati_dataset_testing_noprice.csv', low_memory=False)
test_df.head()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...


In [43]:
# transformamos atributos categoricos
test_df['place_name'] = encoder.fit_transform(test_df[['place_name']])
test_df['state_name'] = encoder.fit_transform(test_df[['state_name']])
test_df['place_with_parent_names'] = encoder.fit_transform(test_df[['place_with_parent_names']])
test_df['property_type'] = encoder.fit_transform(test_df[['property_type']])
test_df['created_on'] = encoder.fit_transform(test_df[['created_on']])
test_df['country_name'] = encoder.fit_transform(test_df[['country_name']])

X_test_df = test_df[['created_on','property_type','place_name','place_with_parent_names',\
              'country_name','state_name','lat','lon','surface_total_in_m2','surface_covered_in_m2',\
              'floor','rooms']]

# completamos valores nan
from sklearn.preprocessing import Imputer
imputer_mean = Imputer(missing_values='NaN', strategy='mean', axis=0)
X_test_df['floor'] = X_test_df[['floor']].fillna(1)
X_test_df['rooms'] = X_test_df[['rooms']].fillna(1)

X_test_df["surface_total_in_m2"] = imputer_mean.fit_transform(X_test_df[["surface_total_in_m2"]])
X_test_df["surface_covered_in_m2"] = imputer_mean.fit_transform(X_test_df[["surface_covered_in_m2"]])
X_test_df["lat"] = imputer_mean.fit_transform(X_test_df[["lat"]])
X_test_df["lon"] = imputer_mean.fit_transform(X_test_df[["lon"]])

X_test_std_df = scaler.transform(X_test_df)

/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:17:

In [44]:
model = RandomForestRegressor(n_estimators=3000, max_features='auto', max_depth=10,min_samples_leaf=5,min_samples_split=10 )
model.fit(X, y)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [46]:
# prediccion model 1
prediction1 = model.predict(X_test_std_df)
X_test_df.head(10)

,created_on,property_type,place_name,place_with_parent_names,country_name,state_name,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms
0,23,1,152,219,0,4,-34.610988,-58.363464,0.0,359.471588,1.0,1.0
1,24,1,29,180,0,3,-34.629923,-58.465820,0.0,359.471588,1.0,1.0
2,0,1,136,210,0,4,-34.589363,-58.412880,53.0,48.000000,1.0,1.0
3,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0
4,0,1,40,193,0,4,-34.629923,-58.465820,51.0,51.000000,1.0,1.0
5,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0
6,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0
7,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0
8,0,1,47,195,0,4,-34.578721,-58.457095,67.0,67.000000,1.0,1.0
9,0,1,7,184,0,4,-34.629923,-58.465820,47.0,47.000000,1.0,1.0


In [47]:
X_fulltrain_scaled = scaler.transform(X)
# Creando modelo con hiper-parametros encontrados
model_std = RandomForestRegressor(n_estimators=3000, max_features='auto', max_depth=100,min_samples_leaf=5,min_samples_split=10 )
model_std.fit(X_fulltrain_scaled, y)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [48]:
# prediccion model 2
prediction2 = model_std.predict(X_test_std_df)
X_test_df.head(10)

,created_on,property_type,place_name,place_with_parent_names,country_name,state_name,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms
0,23,1,152,219,0,4,-34.610988,-58.363464,0.0,359.471588,1.0,1.0
1,24,1,29,180,0,3,-34.629923,-58.465820,0.0,359.471588,1.0,1.0
2,0,1,136,210,0,4,-34.589363,-58.412880,53.0,48.000000,1.0,1.0
3,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0
4,0,1,40,193,0,4,-34.629923,-58.465820,51.0,51.000000,1.0,1.0
5,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0
6,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0
7,0,1,40,193,0,4,-34.629923,-58.465820,39.0,39.000000,1.0,1.0
8,0,1,47,195,0,4,-34.578721,-58.457095,67.0,67.000000,1.0,1.0
9,0,1,7,184,0,4,-34.629923,-58.465820,47.0,47.000000,1.0,1.0


In [49]:
prediction  = (prediction1 + prediction2) / 2
print(prediction)
print(prediction1)
print(prediction2)

[ 429444.7137888   432479.45859003  209025.68256891 ...,  773986.55619567
  196063.23310613  618656.02848883]
[ 235601.27214387  235867.04457643  235601.27214387 ...,  231168.0135398
  235607.275349    231168.0135398 ]
[  623288.15543372   629091.87260364   182450.09299395 ...,
  1316805.09885154   156519.19086325  1006144.04343786]


In [50]:
output = pd.DataFrame( data={"id":test_df["id"], "price_usd": prediction} )

In [51]:
# guardamos
output.to_csv( "../data/result/result_randomforestv02_"+str(now)+".csv", index=False, quoting=3 )